Nella cartella *files* trovi 8 file:
- 2 file di testo
- 2 file audio
- 4 immagini,

con varie estensioni.

**Step 1**

Inizia creando, in un notebook, uno script Python che iteri in ordine alfabetico sui file della cartella files e, a seconda del tipo (audio, documento, immagine), li sposti nella relativa sottocartella (qui sotto trovi un esempio). Se la sottocartella non esiste, il tuo script dovrà crearla automaticamente.

Durante il ciclo, lo script deve stampare le informazioni dei file: nome, tipo e dimensione in byte. Questo è l'output desiderato:

In [ ]:
import os
import shutil
import csv

FILES_FOLDER = "files"
OUTPUT_CSV = "recap.csv"

# gets the extension of a file, returns None if there is no extension
def get_extension(filename):
    if filename.count(".") == 0:
        return None
    return filename.split(".")[-1]

# gets the type of file based on its extension
def get_type(filename):
    extension = get_extension(filename)
    if extension == "mp3":
        return "audio"
    if extension in ["odt", "txt"]:
        return "docs"
    if extension in ["jpg", "jpeg", "png"]:
        return "images"
    return None

# utility function to load the CSV file
def load_csv(filepath):
    data = []
    if os.path.exists(filepath):
        with open(filepath, "r") as file:
            reader = csv.reader(file)
            for row in reader:
                data.append(row)
    return data

# create the directories if they don't exist
for directory in ["files/audio", "files/docs", "files/images"]:
    if not os.path.exists(directory):
        os.mkdir(directory)

Oltre a stamparne le informazioni via via che li sposti, tieni traccia dei file creando un documento *recap.csv* con le stesse informazioni. Trovi un esempio in questa cartella.

La struttura finale della cartella files dovrà essere:

        - files            
            - audio
                - song1.mp3
                - song2.mp3
            - docs
                - ciao.txt
                - pippo.odt
            - images
                - bw.png
                - daffodil.jpg
                - eclipse.png
                - trump.jpeg    
            - recap.csv

Commenta il codice con i passaggi che fai. Questo vale anche per i prossimi Step.

**Attenzione**: lo script, ogni volta che viene lanciato per spostare nuovi file, deve *aggiornare* (e non sovrascrivere) le sottocartelle e il file di recap. Per controllare che tutto funzioni correttamente, puoi aggiungere altri file alla cartella files e fare un test; oppure, puoi dividere gli 8 file originali in due gruppi e lasciarne uno per il test.

**Consiglio**: puoi usare le librerie *os*, *shutil* e *csv*. 
                
---

**Step 2**

Inserisci lo script che hai creato in un piccolo eseguibile (chiamalo *addfile.py* e posizionalo in questa cartella, a fianco del notebook) dotato di *interfaccia a linea di comando* (CLI).

Lo scopo dell'eseguibile è spostare un *singolo* file (che si trova nella cartella files) nella sottocartella di competenza, aggiornando il recap.

L'interfaccia dell'eseguibile ha come unico argomento (obbligatorio) il nome del file da spostare (comprensivo di formato, es: 'trump.jpeg'). Nel caso in cui il file passato come argomento non esista, l'interfaccia deve comunicarlo all'utente.

**Consiglio**: oltre alle precedenti, puoi usare le librerie *sys* e *argparse*.

---

**Step 3**

Una immagine in scala di grigio ha un solo livello di colore, una RGB ne ha 3, una RGBA 4 (l'ultimo è detto canale *alpha*).

Il modulo *Image* della libreria *PIL* permette di caricare un'immagine, che può essere trasformata in un array NumPy attraverso la funzione *np.array*. A partire da tale array, è possibile capire se l'immagine caricata è in scala di grigio, RGB o RGBA.

Aggiungi al notebook dello Step 1 uno script che iteri sulla sottocartella *images* e costruisca una tabella riassuntiva come questa (prodotta con la libreria *tabulate*):

In [43]:
from PIL import Image
import numpy as np
import tabulate

# get mean of the grayscale values, if the image is color, returns 0
def get_grayscale_mean(img_arr):
    if len(img_arr.shape) == 2:
        return np.mean(img_arr)
    else:
        return 0

# get mean of the color values, if the image is grayscale, returns 0
def get_color_mean(img_arr, idx):
    if len(img_arr.shape) == 2:
        return 0
    elif img_arr.shape[2] > idx:
        return np.mean(img_arr[:,:,idx])
    else:
        return 0 
    
# get the mean of the color levels of an image
def get_color_levels_mean(img_arr):
    return {
        "Grayscale": get_grayscale_mean(img_arr),
        "R": get_color_mean(img_arr, 0),
        "G": get_color_mean(img_arr, 1),
        "B": get_color_mean(img_arr, 2),
        "A": get_color_mean(img_arr, 3),
    }

IMAGES_FOLDER = f"{FILES_FOLDER}/images"
images = os.listdir(IMAGES_FOLDER)
data = []

# process each image
for image in images:
    img = Image.open(f"{IMAGES_FOLDER}/{image}")
    img_data = get_color_levels_mean(np.array(img))
    img_data["name"] = image
    img_data["height"] = img.height
    img_data["width"] = img.width
    data.append(img_data)

print(tabulate.tabulate(data, headers="keys", tablefmt="pretty"))

+--------------------+--------------------+--------------------+-------------------+-------------------+--------------+--------+-------+
|     Grayscale      |         R          |         G          |         B         |         A         |     name     | height | width |
+--------------------+--------------------+--------------------+-------------------+-------------------+--------------+--------+-------+
|         0          | 109.25114626865671 | 85.55517611940299  | 4.967289552238806 |         0         | daffodil.jpg |  500   |  335  |
|         0          | 97.01080973671138  | 98.99407848981619  | 90.9237953303527  |         0         |  trump.jpeg  |  183   |  275  |
| 21.480064392089844 |         0          |         0          |         0         |         0         |    bw.png    |  512   |  512  |
|         0          | 109.04953002929688 | 109.52186584472656 | 39.8489990234375  | 133.5909423828125 | eclipse.png  |  256   |  256  |
+--------------------+-------------------

Oltre al nome del file, la tabella riporta:

- altezza dell'immagine, in pixel
- larghezza dell'immagine, in pixel
- se l'immagine è in scala di grigio, la colonna *grayscale* indica la media dei valori dell'unico livello di colore
- se l'immagine è a colori, le altre colonne indicano la media dei valori di ogni livello di colore.

---

**Dovrai consegnare**:
- un notebook con gli Step 1 e 3; per semplicità puoi chiamarlo come questo
- addfile.py con quanto richiesto dallo Step 2.